<p style="text-align: justify">We present here an example illustrating the potential of MEvoLib following the workflow presented in Figure 1. Our study is focused on the molecular evolution of two hmtDNA genes: <em>MT-ATP6</em> and <em>MT-ATP8</em>. We want to know if they have evolved similarly based on the following information: <strong>i)</strong> both genes encode proteins that belong to the same subunit of the ATP synthase enzyme; and, <strong>ii)</strong> these two genes are located in the same strand and they have an overlapping of 42bp.</p>
&nbsp;
&nbsp;
<img src="http://zaramit.unizar.es/download/documents/figure1.png" alt="Phylogenetic inference workflow" width="90%" />
<p style="text-align: justify; margin-left: 3em; margin-right: 3em"><strong>Figure 1:</strong> A common molecular evolution workflow: data fetching, clustering, multialignment, phylogenetic inference and building of the consensus tree.</p>
&nbsp;
&nbsp;
<p style="text-align: justify">First, we are going to download all the hmtDNA sequences from GenBank that have any information related with the <em>MT-ATP6</em> and <em>MT-ATP8</em> genes. The basic query to achieve this purpose would be:
<pre>```"homo sapiens"[porgn] AND mitochondrion[Filter] NOT mRNA[Filter] AND (atp6 OR atp8)```</pre>
But these genes are referred with different nomenclatures in the GenBank metadata. Therefore, our fetching source code is:</p>

In [ ]:
from MEvoLib.Fetch.BioSeqs import BioSeqs

seq_db = BioSeqs.from_entrez(email='eg@test.com', entrez_db='nuccore',
             query='"homo sapiens"[porgn] AND mitochondrion[Filter] NOT mRNA[Filter] AND (atp6 OR atpase6 ' \
                   'OR "atpase 6" OR "atp synthase 6" OR "atpase subunit 6" OR "atp synthetase subunit 6" ' \
                   'OR "atp synthase f0 subunit 6" OR "atp synthase fo subunit 6" OR atp8 OR atpase8 OR ' \
                   '"atpase 8" OR "atp synthase 8" OR "atpase subunit 8" OR "atp synthetase subunit 8" ' \
                   'OR "atp synthase f0 subunit 8" OR "atp synthase fo subunit 8")')
seq_db.write('hmtDNA_ATP6.8_full.gb')
print(seq_db.statistics())

<p style="text-align: justify">We would download 32548 sequences with the first query, whilst the one we have written in the source code fetches 32626 sequences (on 07/Jul/2016). We know two hmtDNA sequences (DQ862537.1 and FR695060.1) that have errors in their biological information, puzzling the genes' clustering of our target genes. Hence, we are going to remove them from the downloaded dataset:</p>

In [ ]:
for accession in ['DQ862537.1', 'FR695060.1'] :
    del seq_db.data[accession]
seq_db.write('hmtDNA_ATP6.8_curated.gb')
print(seq_db.statistics())

<p style="text-align: justify">Next, we need to extract the genes we are interested in from the filtered dataset. For this example, we are going to focus exclusively on the CDS feature. In a complete study, we would also require the <em>gene</em> feature to include those sequences that might have only tagged the genes under this feature. The following source code presents the gene extraction applying the <em>Genes</em> method:</p>

In [ ]:
from MEvoLib import Cluster

gene_dict = Cluster.get_subsets('genes', 'hmtDNA_ATP6.8_curated.gb', 'gb', ['CDS'], None, None,
                                'hmtDNA_ATP6.8_curated.log')
results = {'atp6': [], 'atp8': []}
for key, value in gene_dict.items() :
    if ( (len(value) > 0) and ('atp' in key) ) :
        if ( '6' in key ) :
            results['atp6'].extend(value)
        else : # '8' in key
            results['atp8'].extend(value)

<p style="text-align: justify">We have saved the log file to be able to check all those pairs of terms that our method has merged that do not have enough statistical representation in the dataset. The <em>Genes</em> method only uses the metadata information provided to match all the terms of a single gene. Afterwards, we extract those elements in the dictionary that refer to the <em>MT-ATP6</em> and <em>MT-ATP8</em> genes and we save the corresponding sequence fragments in two separated FASTA files. We are going to build the phylogenetic trees with FastTree and the <em>GTR+CAT</em> evolution model. To do so, we need to align the gene sequences first. Thus, the following code implements the multialignment procedure using Mafft in its default configuration (<em>--auto</em>) and the phylogenetic inference of each gene:</p>

In [ ]:
from Bio import SeqIO
from MEvoLib import Align, Inference

for key, value in results.items() :
    SeqIO.write(value, '{}.fasta'.format(key), 'fasta')
    Align.get_alignment('mafft', '{}.fasta'.format(key), 'fasta', args = 'default',
                        outfile = '{}.aln'.format(key), outfile_format = 'fasta')
    tree, score = Inference.get_phylogeny('fasttree', '{}.aln'.format(key), 'fasta',
                                          args = 'default', outfile = '{}.newick'.format(key),
                                          outfile_format = 'newick', bootstraps = 0)
    print('{}: {}'.format(key, score))

<p style="text-align: justify">As we can see, the resultant phylogeny is returned and saved in a NEWICK file, and its maximum likelihood score is also returned. The inference workflow for this research would be completed with these last results. For instance, to extend our study to include the last stage of the workflow shown in Figure 1, we could be interested in analyzing the consensus tree composed from the <em>MT-ATP6</em> phylogenies of different species, using the Consense method from PHYLIP in its default configuration (<em>majority rule consensus</em>):</p>

<pre>
```python
from MEvoLib import PhyloAssembly

PhyloAssemble.get_consensus_tree('consense', 'atps.newick', 'newick', args = 'default',
                                 outfile = 'cons/atps.newick', outfile_format = 'newick')
```
</pre>